In [6]:
import os
os.environ["MANTLE"] = "lattice"
import silica as si
import magma as m

@si.coroutine(inputs={"I": si.Bit})
def TFF(init=False):
    value = init
    O = value
    while True:
        I = yield O
        O = value
        value = I ^ O
        
tff = si.compile(TFF(), file_name="build/silica_tff.py")
        
with open("build/silica_tff.py", "r") as magma_file:
    print(magma_file.read())

from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator


@cache_definition
def DefineSilicaMux(height, width):
    if "one-hot" == "one-hot":
        if width is None:
            T = Bit
        else:
            T = Bits(width)
        inputs = []
        for i in range(height):
            inputs += [f"I{i}", In(T)]
        class OneHotMux(Circuit):
            name = "SilicaOneHotMux{}{}".format(height, width)
            IO = inputs + ["S", In(Bits(height)), "O", Out(T)]
            @classmethod
            def definition(io):
                or_ = Or(height, width)
                wire(io.O, or_.O)
                for i in range(height):
                    and_ = And(2, width)
                    wire(and_.I0, getattr(io, f"I{i}"))
                    if width is not None:
                        for j in range(width):
                            wire(and_.I1[j], io.S[i])
                    else:
   

In [7]:
!magma -o verilog -m lattice -t TFF build/silica_tff.py
!yosys -p 'synth_ice40 -top TFF -blif build/silica_tff.blif' build/silica_tff.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/silica_tff.txt build/silica_tff.blif
!icetime -tmd hx1k build/silica_tff.txt | grep -B 2 "Total path delay"

import mantle lattice ice40
import mantle lattice mantle40
compiling XOr2
compiling TFF
2.27. Printing statistics.

=== TFF ===

   Number of wires:                  8
   Number of wire bits:              9
   Number of public wires:           8
   Number of public wire bits:       9
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                  2
     SB_DFF                          1
     SB_LUT4                         1

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/silica_tff.blif...
prune...
instantiate_io...
pack...

After packing:
IOs          3 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          1 / 1280
  DFF        1
  CARRY      0
  CARRY, DFF 0
  DFF PASS   0
  CARRY PASS 0
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints

In [12]:
from magma.testing.coroutine import check

@si.coroutine
def inputs_generator():
    while True:
        for i in range(5):
            I = True
            yield I
        for i in range(3):
            I = False
            yield I
            
check(tff, TFF(), 8, inputs_generator())
print("Passed!")

Passed!


In [15]:
from mantle import DFF

magma_TFF = m.DefineCircuit('TFF', 'I', m.In(m.Bit), 'O', m.Out(m.Bit), 'CLK', m.In(m.Clock))

# create ff holding state first
ff = DFF()

# compute the next state
ff( magma_TFF.I ^ ff.O )

m.wire(ff.O, magma_TFF.O)
m.wire(magma_TFF.CLK, ff.CLK)

m.EndDefine()

m.compile("build/magma_tff", magma_TFF)

check(magma_TFF, TFF(), 8, inputs_generator())
print("Passed!")

compiling XOr2
compiling TFF
Passed!


In [16]:
!yosys -p 'synth_ice40 -top TFF -blif build/magma_tff.blif' build/magma_tff.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/magma_tff.txt build/magma_tff.blif
!icetime -tmd hx1k build/magma_tff.txt | grep -B 2 "Total path delay"

2.27. Printing statistics.

=== TFF ===

   Number of wires:                  8
   Number of wire bits:              9
   Number of public wires:           8
   Number of public wire bits:       9
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                  2
     SB_DFF                          1
     SB_LUT4                         1

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/magma_tff.blif...
prune...
instantiate_io...
pack...

After packing:
IOs          3 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          1 / 1280
  DFF        1
  CARRY      0
  CARRY, DFF 0
  DFF PASS   0
  CARRY PASS 0
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promoted 0 nets
  0 globals
realize_constants...
place...
  init